# Refazendo imports e manipulação dos dados

In [45]:
import pandas as pd
import numpy as np
import sklearn as sk
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split


dataset = pd.read_csv('db/movie_dataset.csv', sep=',',  index_col=0, na_values='?')

dataset = dataset.loc[:, ~dataset.columns.isin(['original_title', 'overview', 'tagline', 'title', 'homepage', 'crew', 'keywords', 'original_language', 'id'])]
dataset = dataset.loc[dataset['status'] == 'Released']

dataset_base = dataset.loc[:, dataset.columns.isin(['budget', 'revenue', 'popularity', 'runtime', 'vote_average', 'vote_count'])]
noise = np.random.randint(0, 101, size=dataset_base['revenue'].shape[0])
dataset_base['revenue'] = dataset_base['revenue'] + noise

dataset_base_rev_class = pd.qcut(dataset_base['revenue'], q=4, labels=['very low', 'low', 'medium', 'high'])
dataset_base['revenue'] = dataset_base_rev_class

dataset_dummies = dataset
dataset_dummies = pd.get_dummies(dataset_dummies)
dataset_dummies['revenue'] = dataset_dummies['revenue'] +noise
dataset_dummies_rev = pd.qcut(dataset_dummies['revenue'], q=4, labels=['very low', 'low', 'medium', 'high'])
dataset_dummies['revenue'] = dataset_dummies_rev

/tmp/ipykernel_31462/385667454.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_base['revenue'] = dataset_base['revenue'] + noise
/tmp/ipykernel_31462/385667454.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_base['revenue'] = dataset_base_rev_class


# Grid Search K-Neighbours

In [46]:
dataset_base = dataset_base.loc[:, dataset_base.columns != 'vote_average']
dataset_base = dataset_base.loc[:, dataset_base.columns != 'runtime']

rev_pred_paramX = dataset_base.loc[:, dataset_base.columns != 'revenue'] 
rev_pred_paramY = np.array(dataset_base.loc[:, dataset_base.columns == 'revenue']).ravel()

X_train, X_test, y_train, y_test = train_test_split(rev_pred_paramX, rev_pred_paramY, test_size=0.10, stratify=rev_pred_paramY, random_state=42)

rev_pred_paramX.fillna(1, inplace=True)
rev_pred_paramY = np.nan_to_num(rev_pred_paramY, nan=1)

/tmp/ipykernel_31462/124007373.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rev_pred_paramX.fillna(1, inplace=True)


In [47]:
parameters = {'n_neighbors' : [1, 3, 5, 7, 9, 11, 13], 'weights' : ['uniform', 'distance']}

knn = KNeighborsClassifier()

gs = GridSearchCV(knn, parameters, cv=10, scoring='accuracy')
gs.fit(rev_pred_paramX, rev_pred_paramY)

results = pd.DataFrame(gs.cv_results_)
view = ['params', 'mean_test_score', 'std_test_score', 'rank_test_score']
results[view].sort_values(by='rank_test_score')

,params,mean_test_score,std_test_score,rank_test_score
4,"{'n_neighbors': 5, 'weights': 'uniform'}",0.361785,0.054483,1
6,"{'n_neighbors': 7, 'weights': 'uniform'}",0.355742,0.046821,2
10,"{'n_neighbors': 11, 'weights': 'uniform'}",0.354676,0.072786,3
0,"{'n_neighbors': 1, 'weights': 'uniform'}",0.354503,0.039322,4
1,"{'n_neighbors': 1, 'weights': 'distance'}",0.354503,0.039322,4
2,"{'n_neighbors': 3, 'weights': 'uniform'}",0.354081,0.046866,6
8,"{'n_neighbors': 9, 'weights': 'uniform'}",0.353015,0.065724,7
5,"{'n_neighbors': 5, 'weights': 'distance'}",0.351992,0.048641,8
3,"{'n_neighbors': 3, 'weights': 'distance'}",0.349493,0.048084,9
7,"{'n_neighbors': 7, 'weights': 'distance'}",0.348654,0.048555,10


In [48]:
dataset_dummies = dataset_dummies.loc[:, dataset_dummies.columns != 'vote_average']
dataset_dummies = dataset_dummies.loc[:, dataset_dummies.columns != 'runtime']

rev_pred_paramX = dataset_dummies.loc[:, dataset_dummies.columns != 'revenue'] 
rev_pred_paramY = np.array(dataset_dummies.loc[:, dataset_dummies.columns == 'revenue']).ravel()

X_train, X_test, y_train, y_test = train_test_split(rev_pred_paramX, rev_pred_paramY, test_size=0.10, stratify=rev_pred_paramY, random_state=42)

rev_pred_paramX.fillna(1, inplace=True)
rev_pred_paramY = np.nan_to_num(rev_pred_paramY, nan=1)

/tmp/ipykernel_31462/727090146.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rev_pred_paramX.fillna(1, inplace=True)


In [49]:
parameters = {'n_neighbors' : [1, 3, 5, 7, 9, 11, 13], 'weights' : ['uniform', 'distance']}

knn = KNeighborsClassifier()

gs = GridSearchCV(knn, parameters, cv=10, scoring='accuracy')
gs.fit(rev_pred_paramX, rev_pred_paramY)

results = pd.DataFrame(gs.cv_results_)
view = ['params', 'mean_test_score', 'std_test_score', 'rank_test_score']
results[view].sort_values(by='rank_test_score')

,params,mean_test_score,std_test_score,rank_test_score
4,"{'n_neighbors': 5, 'weights': 'uniform'}",0.362829,0.053583,1
7,"{'n_neighbors': 7, 'weights': 'distance'}",0.358247,0.050634,2
9,"{'n_neighbors': 9, 'weights': 'distance'}",0.358037,0.050999,3
5,"{'n_neighbors': 5, 'weights': 'distance'}",0.357831,0.049234,4
6,"{'n_neighbors': 7, 'weights': 'uniform'}",0.356160,0.046607,5
11,"{'n_neighbors': 11, 'weights': 'distance'}",0.356157,0.053088,6
10,"{'n_neighbors': 11, 'weights': 'uniform'}",0.355093,0.072978,7
8,"{'n_neighbors': 9, 'weights': 'uniform'}",0.353642,0.065222,8
13,"{'n_neighbors': 13, 'weights': 'distance'}",0.351362,0.050172,9
0,"{'n_neighbors': 1, 'weights': 'uniform'}",0.350744,0.046400,10
